# Determine derivative of Jacobian from angular velocity to exponential rates

Peter Corke 2021

SymPy code to deterine the time derivative of the mapping from angular velocity to exponential coordinate rates.

In [2]:
from sympy import *

A rotation matrix can be expressed in terms of exponential coordinates (also called Euler vector)

$
\mathbf{R} = e^{[\varphi]_\times} 
$
where $\mathbf{R} \in SO(3)$ and $\varphi \in \mathbb{R}^3$.

The mapping from angular velocity $\omega$ to exponential coordinate rates $\dot{\varphi}$ is

$
\dot{\varphi} = \mathbf{A} \omega
$

where $\mathbf{A}$ is given by (2.107) of [Robot Dynamics Lecture Notes, Robotic Systems Lab, ETH Zurich, 2018](https://ethz.ch/content/dam/ethz/special-interest/mavt/robotics-n-intelligent-systems/rsl-dam/documents/RobotDynamics2018/RD_HS2018script.pdf)


$
\mathbf{A} = \mathbf{1}_{3 \times 3} - \frac{1}{2} [v]_\times + [v]^2_\times \frac{1}{\theta^2} \left( 1 - \frac{\theta}{2} \frac{\sin \theta}{1 - \cos \theta} \right)
$
where $\theta = \| \varphi \|$ and $v = \hat{\varphi}$

We simplify the equation as

$
\mathbf{A} = \mathbf{1}_{3 \times 3} - \frac{1}{2} [v]_\times + [v]^2_\times \Theta
$

where
$
\Theta = \frac{1}{\theta^2} \left( 1 - \frac{\theta}{2} \frac{\sin \theta}{1 - \cos \theta} \right)
$

We can find the derivative using the chain rule

$
\dot{\mathbf{A}} = - \frac{1}{2} [\dot{v}]_\times + 2 [v]_\times [\dot{v}]_\times \Theta + [v]^2_\times \dot{\Theta}
$

We start by defining some symbols

In [21]:
Theta, theta, theta_dot, t = symbols('Theta theta theta_dot t', real=True)

We start by finding an expression for $\Theta$ which depends on $\theta(t)$

In [22]:
theta_t = Function(theta)(t)

In [23]:
Theta = 1 / theta_t ** 2 * (1 - theta_t / 2 * sin(theta_t) / (1 - cos(theta_t)))
Theta

(1 - theta(t)*sin(theta(t))/(2*(1 - cos(theta(t)))))/theta(t)**2

and now determine the derivative

In [24]:
T_dot = Theta.diff(t)
T_dot

-2*(1 - theta(t)*sin(theta(t))/(2*(1 - cos(theta(t)))))*Derivative(theta(t), t)/theta(t)**3 + (-theta(t)*cos(theta(t))*Derivative(theta(t), t)/(2*(1 - cos(theta(t)))) - sin(theta(t))*Derivative(theta(t), t)/(2*(1 - cos(theta(t)))) + theta(t)*sin(theta(t))**2*Derivative(theta(t), t)/(2*(1 - cos(theta(t)))**2))/theta(t)**2

which is a somewhat complex expression that depends on $\theta(t)$ and $\dot{\theta}(t)$.

We will remove the time dependency and generate code

In [25]:
T_dot = T_dot.subs([(theta_t.diff(t), theta_dot), (theta_t, theta)])

In [26]:
pycode(T_dot)

'(-1/2*theta*theta_dot*math.cos(theta)/(1 - math.cos(theta)) + (1/2)*theta*theta_dot*math.sin(theta)**2/(1 - math.cos(theta))**2 - 1/2*theta_dot*math.sin(theta)/(1 - math.cos(theta)))/theta**2 - 2*theta_dot*(-1/2*theta*math.sin(theta)/(1 - math.cos(theta)) + 1)/theta**3'

In order to evaluate the line above we need an expression for $\theta$ and $\dot{\theta}$.  $\theta$ is the norm of $\varphi$ whose elements are functions of time

In [38]:
phi_names = ('varphi_0', 'varphi_1', 'varphi_2')
phi = []  # names of angles, eg. theta
phi_t = []  # angles as function of time, eg. theta(t)
phi_d = []  # derivative of above, eg. d theta(t) / dt
phi_n = []  # symbol to represent above, eg. theta_dot
for i in phi_names:
    phi.append(symbols(i, real=True))
    phi_t.append(Function(phi[-1])(t))
    phi_d.append(phi_t[-1].diff(t))
    phi_n.append(i + '_dot')

Compute the norm

In [40]:
theta = Matrix(phi_t).norm()
theta

sqrt(varphi_0(t)**2 + varphi_1(t)**2 + varphi_2(t)**2)

and find its derivative

In [41]:
theta_dot = theta.diff(t)
theta_dot

(varphi_0(t)*Derivative(varphi_0(t), t) + varphi_1(t)*Derivative(varphi_1(t), t) + varphi_2(t)*Derivative(varphi_2(t), t))/sqrt(varphi_0(t)**2 + varphi_1(t)**2 + varphi_2(t)**2)

and now remove the time dependenices

In [42]:
theta_dot = theta_dot.subs(a for a in zip(phi_d, phi_n))
theta_dot = theta_dot.subs(a for a in zip(phi_t, phi))
theta_dot

(varphi_0*varphi_0_dot + varphi_1*varphi_1_dot + varphi_2*varphi_2_dot)/sqrt(varphi_0**2 + varphi_1**2 + varphi_2**2)

which is simply the dot product over the norm.

In [7]:
A, t = symbols('A t', real=True)
A_t = Function(A)(t)
d = diff(exp(A_t), t)
print(d)

exp(A(t))*Derivative(A(t), t)
